In [56]:
##################
# Import libraries
##################

import pandas as pd
import numpy as np
import math


###########
# Load data
###########

iris = pd.read_csv('iris.csv', header=None)
iris.columns = ['sepal_length', 'sepal_width',
                'petal_length', 'petal_width', 'target_class']


###############################################
# Merging rows based on least chi square values 
###############################################

def merge_rows(df,feature):

    tdf = df[:-1]
    distinct_values = sorted(set(tdf['chi2']), reverse=False)

    col_names =  [feature,'Iris-setosa', 'Iris-versicolor', 
                  'Iris-virginica','chi2']
    #new dataframe to send back
    updated_df  = pd.DataFrame(columns = col_names) 
    
    updated_df_index=0
    for index, row in df.iterrows(): #iterating over old dataframe
        if(index==0):
            updated_df.loc[len(updated_df)] = df.loc[index]
            updated_df_index+=1
        else:
            if(df.loc[index-1]['chi2']==distinct_values[0]): #merge
                updated_df.loc[updated_df_index-1]['Iris-setosa']\
                +=df.loc[index]['Iris-setosa']
                updated_df.loc[updated_df_index-1]['Iris-versicolor']\
                +=df.loc[index]['Iris-versicolor']
                updated_df.loc[updated_df_index-1]['Iris-virginica']\
                +=df.loc[index]['Iris-virginica']
            else:
                updated_df.loc[len(updated_df)] = df.loc[index]
                updated_df_index+=1
                
    updated_df['chi2'] = 0.   #clearing old chi square values

    return updated_df
        

#####################
# Chi square function
#####################

def calc_chi2(array):
    shape = array.shape
    n = float(array.sum()) #total number of entries
    row={}
    column={}
    
    #find row-wise summations
    for i in range(shape[0]):
        row[i] = array[i].sum()
    
    #find column-wise summations
    for j in range(shape[1]):
        column[j] = array[:,j].sum()

    chi2 = 0
    
    #using the chi square formula
    for i in range(shape[0]):
        for j in range(shape[1]):
            eij = row[i]*column[j] / n 
            oij = array[i,j]  
            if eij==0.:
                chi2 += 0. #making sure nan doesnt bother us
            else: 
                chi2 += math.pow((oij - eij),2) / float(eij)
  
    return chi2
    
    
##################################################################
# This function calculates the chi square values for each row pair
##################################################################

def update_chi2_column(contingency_table,feature):
    
    for index, row in contingency_table.iterrows():
        #we dont wanna work with the very last row alone
        if(index!=contingency_table.shape[0]-1): 
            
            # prepare an array with two rows of data at a time
            list1=[]
            list2=[]
            list1.append(contingency_table.loc[index]['Iris-setosa'])
            list1.append(contingency_table.loc[index]['Iris-versicolor'])
            list1.append(contingency_table.loc[index]['Iris-virginica'])
            list2.append(contingency_table.loc[index+1]['Iris-setosa'])
            list2.append(contingency_table.loc[index+1]['Iris-versicolor'])
            list2.append(contingency_table.loc[index+1]['Iris-virginica'])
            prep_chi2 = np.array([np.array(list1),np.array(list2)])
            
            #actually compute the chi square values
            c2 = calc_chi2(prep_chi2)
            
            #update dataframe
            contingency_table.loc[index]['chi2'] = c2
    return contingency_table


##############################################
# This function calculates the frequency table
##############################################

def create_contingency_table(dataframe,feature):
    distinct_values = sorted(set(dataframe[feature]), reverse=False)
    col_names =  [feature,'Iris-setosa', 'Iris-versicolor', \
                  'Iris-virginica','chi2']
    my_contingency  = pd.DataFrame(columns = col_names)
    
    #these are the distinct attribute values
    for i in range(len(distinct_values)): 
        temp_df=dataframe.loc[dataframe[feature]==distinct_values[i]]
        count_dict = temp_df["target_class"].value_counts().to_dict()

        #initialize with zero frequencies
        setosa_count = 0
        versicolor_count = 0
        virginica_count = 0
        
        #update if necessary
        if 'Iris-setosa' in count_dict:
            setosa_count = count_dict['Iris-setosa']
        if 'Iris-versicolor' in count_dict:
            versicolor_count = count_dict['Iris-versicolor']
        if 'Iris-virginica' in count_dict:
            virginica_count = count_dict['Iris-virginica']

        new_row = [distinct_values[i],setosa_count,versicolor_count,\
                   virginica_count,0]
        my_contingency.loc[len(my_contingency)] = new_row

    return my_contingency


####################
# Chi Merge function
####################

def chimerge(feature, data=iris, max_interval=6):
    df = data.sort_values(by=[feature],ascending=True).reset_index()
    
    #generate frequency table
    contingency_table = create_contingency_table(df,feature)

    #calculate initial number of intervals. #initially, all entries
    #are intervals on their own
    num_intervals= contingency_table.shape[0] 

    # keep looping till max-interval condition satisfied
    while num_intervals > max_interval: 
        #compute chi square for adjacent row pairs
        chi2_df = update_chi2_column(contingency_table,feature) 
        
        #merge rows based on lowest chi square values
        contingency_table = merge_rows(chi2_df,feature)
        
        #update number of intervals
        num_intervals= contingency_table.shape[0]               

    # Print results
    print('The split points for '+feature+' are:')
    for index, row in contingency_table.iterrows():
        print(contingency_table.loc[index][feature])
    
    print('The final intervals for '+feature+' are:')
    for index, row in contingency_table.iterrows():
        if(index!=contingency_table.shape[0]-1):
            for index2, row2 in df.iterrows():
                if df.loc[index2][feature]<contingency_table.\
                loc[index+1][feature]:
                    temp = df.loc[index2][feature]
        else:
            temp = df[feature].iloc[-1]
        print("["+str(contingency_table.loc[index][feature])+","+str(temp)+"]")
    print(" ")

    
######
# Init
######

for feature in ['sepal_length', 'sepal_width', 'petal_length',\
                'petal_width']:
    chimerge(feature=feature)


The split points for sepal_length are:
4.3
4.9
5.0
5.5
5.8
7.1
The final intervals for sepal_length are:
[4.3,4.8]
[4.9,4.9]
[5.0,5.4]
[5.5,5.7]
[5.8,7.0]
[7.1,7.9]
 
The split points for sepal_width are:
2.0
2.3
2.5
2.9
3.0
3.4
The final intervals for sepal_width are:
[2.0,2.2]
[2.3,2.4]
[2.5,2.8]
[2.9,2.9]
[3.0,3.3]
[3.4,4.4]
 
The split points for petal_length are:
1.0
3.0
4.5
4.8
5.0
5.2
The final intervals for petal_length are:
[1.0,1.9]
[3.0,4.4]
[4.5,4.7]
[4.8,4.9]
[5.0,5.1]
[5.2,6.9]
 
The split points for petal_width are:
0.1
1.0
1.4
1.7
1.8
1.9
The final intervals for petal_width are:
[0.1,0.6]
[1.0,1.3]
[1.4,1.6]
[1.7,1.7]
[1.8,1.8]
[1.9,2.5]
 


In [79]:
arr = range(3000)

In [83]:
count=0
for x in arr:
    print(x)
    if count%100==0:
        print("yes")
    count+=1

0
yes
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46
47
48
49
50
51
52
53
54
55
56
57
58
59
60
61
62
63
64
65
66
67
68
69
70
71
72
73
74
75
76
77
78
79
80
81
82
83
84
85
86
87
88
89
90
91
92
93
94
95
96
97
98
99
100
yes
101
102
103
104
105
106
107
108
109
110
111
112
113
114
115
116
117
118
119
120
121
122
123
124
125
126
127
128
129
130
131
132
133
134
135
136
137
138
139
140
141
142
143
144
145
146
147
148
149
150
151
152
153
154
155
156
157
158
159
160
161
162
163
164
165
166
167
168
169
170
171
172
173
174
175
176
177
178
179
180
181
182
183
184
185
186
187
188
189
190
191
192
193
194
195
196
197
198
199
200
yes
201
202
203
204
205
206
207
208
209
210
211
212
213
214
215
216
217
218
219
220
221
222
223
224
225
226
227
228
229
230
231
232
233
234
235
236
237
238
239
240
241
242
243
244
245
246
247
248
249
250
251
252
253
254
255
256
257
258
259
260
261
262
263
264
265
266
267
268
269
270
271
272
273
27

In [77]:
for x in arr:
    print(np.floor(float(x)/100))

0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
2.0
2.0
2.0
2.0
2.0
2.0
2.0
2.0
2.0
2.0
2.0
2.0
2.0
2.0
2.0
2.0
2.0
2.0
2.0
2.0
2.0
2.0
2.0
2.0
2.0
2.0
2.0
2.0
2.0
2.0
2.0
2.0
2.0
2.0
2.0
2.0
2.0
2.0
2.0
2.0
2.0
2.0
2.0
2.0
2.0
2.0
2.0
2.0
2.0
2.0


37.0
37.0
37.0
37.0
37.0
37.0
37.0
37.0
37.0
37.0
37.0
37.0
37.0
37.0
37.0
37.0
37.0
37.0
37.0
37.0
37.0
37.0
37.0
37.0
37.0
37.0
37.0
37.0
37.0
37.0
37.0
37.0
37.0
37.0
37.0
37.0
37.0
37.0
37.0
37.0
37.0
37.0
37.0
37.0
37.0
37.0
37.0
37.0
37.0
37.0
37.0
37.0
37.0
37.0
37.0
37.0
37.0
37.0
37.0
37.0
37.0
37.0
37.0
37.0
37.0
37.0
37.0
37.0
37.0
37.0
37.0
37.0
37.0
37.0
37.0
37.0
37.0
37.0
37.0
37.0
37.0
38.0
38.0
38.0
38.0
38.0
38.0
38.0
38.0
38.0
38.0
38.0
38.0
38.0
38.0
38.0
38.0
38.0
38.0
38.0
38.0
38.0
38.0
38.0
38.0
38.0
38.0
38.0
38.0
38.0
38.0
38.0
38.0
38.0
38.0
38.0
38.0
38.0
38.0
38.0
38.0
38.0
38.0
38.0
38.0
38.0
38.0
38.0
38.0
38.0
38.0
38.0
38.0
38.0
38.0
38.0
38.0
38.0
38.0
38.0
38.0
38.0
38.0
38.0
38.0
38.0
38.0
38.0
38.0
38.0
38.0
38.0
38.0
38.0
38.0
38.0
38.0
38.0
38.0
38.0
38.0
38.0
38.0
38.0
38.0
38.0
38.0
38.0
38.0
38.0
38.0
38.0
38.0
38.0
38.0
38.0
38.0
38.0
38.0
38.0
38.0
39.0
39.0
39.0
39.0
39.0
39.0
39.0
39.0
39.0
39.0
39.0
39.0
39.0
39.0
39.0
39.0
39.0
39.0
39.0


60.0
60.0
60.0
60.0
60.0
60.0
60.0
60.0
60.0
60.0
60.0
60.0
60.0
60.0
60.0
60.0
60.0
60.0
60.0
60.0
60.0
60.0
60.0
60.0
60.0
60.0
60.0
60.0
60.0
60.0
60.0
60.0
60.0
60.0
60.0
60.0
60.0
60.0
60.0
60.0
60.0
60.0
60.0
60.0
60.0
60.0
60.0
60.0
60.0
60.0
60.0
60.0
60.0
60.0
60.0
60.0
60.0
60.0
60.0
60.0
60.0
60.0
60.0
60.0
60.0
60.0
60.0
60.0
60.0
60.0
60.0
60.0
60.0
60.0
60.0
60.0
60.0
60.0
60.0
60.0
60.0
60.0
60.0
60.0
60.0
60.0
60.0
60.0
60.0
60.0
60.0
60.0
60.0
61.0
61.0
61.0
61.0
61.0
61.0
61.0
61.0
61.0
61.0
61.0
61.0
61.0
61.0
61.0
61.0
61.0
61.0
61.0
61.0
61.0
61.0
61.0
61.0
61.0
61.0
61.0
61.0
61.0
61.0
61.0
61.0
61.0
61.0
61.0
61.0
61.0
61.0
61.0
61.0
61.0
61.0
61.0
61.0
61.0
61.0
61.0
61.0
61.0
61.0
61.0
61.0
61.0
61.0
61.0
61.0
61.0
61.0
61.0
61.0
61.0
61.0
61.0
61.0
61.0
61.0
61.0
61.0
61.0
61.0
61.0
61.0
61.0
61.0
61.0
61.0
61.0
61.0
61.0
61.0
61.0
61.0
61.0
61.0
61.0
61.0
61.0
61.0
61.0
61.0
61.0
61.0
61.0
61.0
61.0
61.0
61.0
61.0
61.0
61.0
62.0
62.0
62.0
62.0
62.0
62.0
62.0


84.0
84.0
84.0
84.0
85.0
85.0
85.0
85.0
85.0
85.0
85.0
85.0
85.0
85.0
85.0
85.0
85.0
85.0
85.0
85.0
85.0
85.0
85.0
85.0
85.0
85.0
85.0
85.0
85.0
85.0
85.0
85.0
85.0
85.0
85.0
85.0
85.0
85.0
85.0
85.0
85.0
85.0
85.0
85.0
85.0
85.0
85.0
85.0
85.0
85.0
85.0
85.0
85.0
85.0
85.0
85.0
85.0
85.0
85.0
85.0
85.0
85.0
85.0
85.0
85.0
85.0
85.0
85.0
85.0
85.0
85.0
85.0
85.0
85.0
85.0
85.0
85.0
85.0
85.0
85.0
85.0
85.0
85.0
85.0
85.0
85.0
85.0
85.0
85.0
85.0
85.0
85.0
85.0
85.0
85.0
85.0
85.0
85.0
85.0
85.0
85.0
85.0
85.0
85.0
86.0
86.0
86.0
86.0
86.0
86.0
86.0
86.0
86.0
86.0
86.0
86.0
86.0
86.0
86.0
86.0
86.0
86.0
86.0
86.0
86.0
86.0
86.0
86.0
86.0
86.0
86.0
86.0
86.0
86.0
86.0
86.0
86.0
86.0
86.0
86.0
86.0
86.0
86.0
86.0
86.0
86.0
86.0
86.0
86.0
86.0
86.0
86.0
86.0
86.0
86.0
86.0
86.0
86.0
86.0
86.0
86.0
86.0
86.0
86.0
86.0
86.0
86.0
86.0
86.0
86.0
86.0
86.0
86.0
86.0
86.0
86.0
86.0
86.0
86.0
86.0
86.0
86.0
86.0
86.0
86.0
86.0
86.0
86.0
86.0
86.0
86.0
86.0
86.0
86.0
86.0
86.0
86.0
86.0
86.0
86.0


108.0
108.0
108.0
108.0
108.0
108.0
108.0
108.0
108.0
108.0
108.0
108.0
108.0
108.0
108.0
108.0
108.0
108.0
108.0
108.0
108.0
108.0
108.0
108.0
108.0
108.0
108.0
108.0
108.0
108.0
108.0
108.0
108.0
108.0
108.0
108.0
108.0
108.0
108.0
108.0
108.0
108.0
109.0
109.0
109.0
109.0
109.0
109.0
109.0
109.0
109.0
109.0
109.0
109.0
109.0
109.0
109.0
109.0
109.0
109.0
109.0
109.0
109.0
109.0
109.0
109.0
109.0
109.0
109.0
109.0
109.0
109.0
109.0
109.0
109.0
109.0
109.0
109.0
109.0
109.0
109.0
109.0
109.0
109.0
109.0
109.0
109.0
109.0
109.0
109.0
109.0
109.0
109.0
109.0
109.0
109.0
109.0
109.0
109.0
109.0
109.0
109.0
109.0
109.0
109.0
109.0
109.0
109.0
109.0
109.0
109.0
109.0
109.0
109.0
109.0
109.0
109.0
109.0
109.0
109.0
109.0
109.0
109.0
109.0
109.0
109.0
109.0
109.0
109.0
109.0
109.0
109.0
109.0
109.0
109.0
109.0
109.0
109.0
109.0
109.0
109.0
109.0
110.0
110.0
110.0
110.0
110.0
110.0
110.0
110.0
110.0
110.0
110.0
110.0
110.0
110.0
110.0
110.0
110.0
110.0
110.0
110.0
110.0
110.0
110.0
110.0
110.

...

196.0
196.0
196.0
196.0
196.0
196.0
196.0
196.0
196.0
196.0
196.0
196.0
196.0
196.0
196.0
196.0
196.0
196.0
196.0
196.0
196.0
196.0
196.0
196.0
196.0
197.0
197.0
197.0
197.0
197.0
197.0
197.0
197.0
197.0
197.0
197.0
197.0
197.0
197.0
197.0
197.0
197.0
197.0
197.0
197.0
197.0
197.0
197.0
197.0
197.0
197.0
197.0
197.0
197.0
197.0
197.0
197.0
197.0
197.0
197.0
197.0
197.0
197.0
197.0
197.0
197.0
197.0
197.0
197.0
197.0
197.0
197.0
197.0
197.0
197.0
197.0
197.0
197.0
197.0
197.0
197.0
197.0
197.0
197.0
197.0
197.0
197.0
197.0
197.0
197.0
197.0
197.0
197.0
197.0
197.0
197.0
197.0
197.0
197.0
197.0
197.0
197.0
197.0
197.0
197.0
197.0
197.0
197.0
197.0
197.0
197.0
197.0
197.0
197.0
197.0
197.0
197.0
197.0
197.0
197.0
197.0
197.0
197.0
197.0
197.0
198.0
198.0
198.0
198.0
198.0
198.0
198.0
198.0
198.0
198.0
198.0
198.0
198.0
198.0
198.0
198.0
198.0
198.0
198.0
198.0
198.0
198.0
198.0
198.0
198.0
198.0
198.0
198.0
198.0
198.0
198.0
198.0
198.0
198.0
198.0
198.0
198.0
198.0
198.0
198.0
198.0
198.

220.0
220.0
220.0
220.0
220.0
220.0
220.0
220.0
220.0
220.0
220.0
220.0
220.0
220.0
220.0
220.0
220.0
220.0
220.0
220.0
220.0
220.0
220.0
220.0
220.0
220.0
220.0
220.0
220.0
220.0
220.0
220.0
220.0
220.0
220.0
220.0
220.0
220.0
220.0
220.0
220.0
220.0
220.0
220.0
220.0
220.0
220.0
220.0
220.0
220.0
220.0
220.0
220.0
220.0
220.0
220.0
220.0
220.0
220.0
220.0
220.0
220.0
220.0
220.0
220.0
220.0
220.0
220.0
220.0
220.0
220.0
220.0
220.0
220.0
220.0
220.0
220.0
220.0
220.0
220.0
220.0
220.0
220.0
220.0
220.0
220.0
220.0
220.0
221.0
221.0
221.0
221.0
221.0
221.0
221.0
221.0
221.0
221.0
221.0
221.0
221.0
221.0
221.0
221.0
221.0
221.0
221.0
221.0
221.0
221.0
221.0
221.0
221.0
221.0
221.0
221.0
221.0
221.0
221.0
221.0
221.0
221.0
221.0
221.0
221.0
221.0
221.0
221.0
221.0
221.0
221.0
221.0
221.0
221.0
221.0
221.0
221.0
221.0
221.0
221.0
221.0
221.0
221.0
221.0
221.0
221.0
221.0
221.0
221.0
221.0
221.0
221.0
221.0
221.0
221.0
221.0
221.0
221.0
221.0
221.0
221.0
221.0
221.0
221.0
221.0
221.0
221.

244.0
244.0
244.0
244.0
244.0
244.0
244.0
244.0
244.0
244.0
244.0
244.0
244.0
244.0
244.0
244.0
244.0
244.0
244.0
244.0
244.0
244.0
244.0
244.0
244.0
244.0
244.0
244.0
244.0
244.0
244.0
244.0
244.0
244.0
244.0
244.0
244.0
244.0
244.0
244.0
244.0
244.0
244.0
244.0
244.0
244.0
244.0
244.0
244.0
244.0
244.0
244.0
244.0
244.0
244.0
244.0
244.0
244.0
244.0
244.0
244.0
244.0
244.0
244.0
244.0
244.0
244.0
244.0
244.0
244.0
244.0
244.0
244.0
244.0
244.0
244.0
244.0
244.0
244.0
244.0
244.0
245.0
245.0
245.0
245.0
245.0
245.0
245.0
245.0
245.0
245.0
245.0
245.0
245.0
245.0
245.0
245.0
245.0
245.0
245.0
245.0
245.0
245.0
245.0
245.0
245.0
245.0
245.0
245.0
245.0
245.0
245.0
245.0
245.0
245.0
245.0
245.0
245.0
245.0
245.0
245.0
245.0
245.0
245.0
245.0
245.0
245.0
245.0
245.0
245.0
245.0
245.0
245.0
245.0
245.0
245.0
245.0
245.0
245.0
245.0
245.0
245.0
245.0
245.0
245.0
245.0
245.0
245.0
245.0
245.0
245.0
245.0
245.0
245.0
245.0
245.0
245.0
245.0
245.0
245.0
245.0
245.0
245.0
245.0
245.0
245.0
245.

264.0
264.0
264.0
264.0
264.0
264.0
264.0
264.0
264.0
264.0
264.0
264.0
264.0
264.0
264.0
264.0
264.0
264.0
264.0
264.0
264.0
264.0
264.0
264.0
264.0
264.0
264.0
264.0
264.0
264.0
264.0
264.0
264.0
264.0
264.0
264.0
264.0
264.0
264.0
264.0
264.0
264.0
264.0
264.0
264.0
264.0
264.0
264.0
264.0
264.0
264.0
264.0
264.0
264.0
264.0
264.0
264.0
264.0
264.0
264.0
264.0
264.0
264.0
264.0
264.0
264.0
264.0
264.0
264.0
264.0
264.0
264.0
264.0
264.0
264.0
264.0
264.0
264.0
264.0
264.0
264.0
264.0
264.0
264.0
264.0
264.0
264.0
264.0
264.0
264.0
264.0
264.0
264.0
264.0
264.0
264.0
264.0
264.0
264.0
265.0
265.0
265.0
265.0
265.0
265.0
265.0
265.0
265.0
265.0
265.0
265.0
265.0
265.0
265.0
265.0
265.0
265.0
265.0
265.0
265.0
265.0
265.0
265.0
265.0
265.0
265.0
265.0
265.0
265.0
265.0
265.0
265.0
265.0
265.0
265.0
265.0
265.0
265.0
265.0
265.0
265.0
265.0
265.0
265.0
265.0
265.0
265.0
265.0
265.0
265.0
265.0
265.0
265.0
265.0
265.0
265.0
265.0
265.0
265.0
265.0
265.0
265.0
265.0
265.0
265.0
265.0
265.

285.0
285.0
285.0
285.0
285.0
285.0
285.0
285.0
285.0
285.0
285.0
285.0
285.0
285.0
285.0
285.0
285.0
285.0
285.0
285.0
285.0
285.0
285.0
285.0
285.0
285.0
285.0
285.0
285.0
285.0
285.0
285.0
285.0
285.0
285.0
285.0
285.0
285.0
285.0
285.0
285.0
285.0
285.0
285.0
285.0
285.0
285.0
285.0
285.0
285.0
285.0
285.0
285.0
285.0
285.0
285.0
285.0
285.0
285.0
285.0
285.0
285.0
285.0
285.0
285.0
285.0
285.0
285.0
285.0
285.0
285.0
285.0
285.0
285.0
285.0
285.0
285.0
285.0
285.0
286.0
286.0
286.0
286.0
286.0
286.0
286.0
286.0
286.0
286.0
286.0
286.0
286.0
286.0
286.0
286.0
286.0
286.0
286.0
286.0
286.0
286.0
286.0
286.0
286.0
286.0
286.0
286.0
286.0
286.0
286.0
286.0
286.0
286.0
286.0
286.0
286.0
286.0
286.0
286.0
286.0
286.0
286.0
286.0
286.0
286.0
286.0
286.0
286.0
286.0
286.0
286.0
286.0
286.0
286.0
286.0
286.0
286.0
286.0
286.0
286.0
286.0
286.0
286.0
286.0
286.0
286.0
286.0
286.0
286.0
286.0
286.0
286.0
286.0
286.0
286.0
286.0
286.0
286.0
286.0
286.0
286.0
286.0
286.0
286.0
286.0
286.0
286.

In [78]:
for x in arr:
    print(np.floor(float(x)/10))

0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
2.0
2.0
2.0
2.0
2.0
2.0
2.0
2.0
2.0
2.0
3.0
3.0
3.0
3.0
3.0
3.0
3.0
3.0
3.0
3.0
4.0
4.0
4.0
4.0
4.0
4.0
4.0
4.0
4.0
4.0
5.0
5.0
5.0
5.0
5.0
5.0
5.0
5.0
5.0
5.0
6.0
6.0
6.0
6.0
6.0
6.0
6.0
6.0
6.0
6.0
7.0
7.0
7.0
7.0
7.0
7.0
7.0
7.0
7.0
7.0
8.0
8.0
8.0
8.0
8.0
8.0
8.0
8.0
8.0
8.0
9.0
9.0
9.0
9.0
9.0
9.0
9.0
9.0
9.0
9.0
10.0
10.0
10.0
10.0
10.0
10.0
10.0
10.0
10.0
10.0
11.0
11.0
11.0
11.0
11.0
11.0
11.0
11.0
11.0
11.0
12.0
12.0
12.0
12.0
12.0
12.0
12.0
12.0
12.0
12.0
13.0
13.0
13.0
13.0
13.0
13.0
13.0
13.0
13.0
13.0
14.0
14.0
14.0
14.0
14.0
14.0
14.0
14.0
14.0
14.0
15.0
15.0
15.0
15.0
15.0
15.0
15.0
15.0
15.0
15.0
16.0
16.0
16.0
16.0
16.0
16.0
16.0
16.0
16.0
16.0
17.0
17.0
17.0
17.0
17.0
17.0
17.0
17.0
17.0
17.0
18.0
18.0
18.0
18.0
18.0
18.0
18.0
18.0
18.0
18.0
19.0
19.0
19.0
19.0
19.0
19.0
19.0
19.0
19.0
19.0
20.0
20.0
20.0
20.0
20.0
20.0
20.0
20.0
20.0
20.0
21.0
21.0
21.0
21.0
21.0
21.0
21.0
21.0
21.0
21.0


376.0
376.0
376.0
376.0
377.0
377.0
377.0
377.0
377.0
377.0
377.0
377.0
377.0
377.0
378.0
378.0
378.0
378.0
378.0
378.0
378.0
378.0
378.0
378.0
379.0
379.0
379.0
379.0
379.0
379.0
379.0
379.0
379.0
379.0
380.0
380.0
380.0
380.0
380.0
380.0
380.0
380.0
380.0
380.0
381.0
381.0
381.0
381.0
381.0
381.0
381.0
381.0
381.0
381.0
382.0
382.0
382.0
382.0
382.0
382.0
382.0
382.0
382.0
382.0
383.0
383.0
383.0
383.0
383.0
383.0
383.0
383.0
383.0
383.0
384.0
384.0
384.0
384.0
384.0
384.0
384.0
384.0
384.0
384.0
385.0
385.0
385.0
385.0
385.0
385.0
385.0
385.0
385.0
385.0
386.0
386.0
386.0
386.0
386.0
386.0
386.0
386.0
386.0
386.0
387.0
387.0
387.0
387.0
387.0
387.0
387.0
387.0
387.0
387.0
388.0
388.0
388.0
388.0
388.0
388.0
388.0
388.0
388.0
388.0
389.0
389.0
389.0
389.0
389.0
389.0
389.0
389.0
389.0
389.0
390.0
390.0
390.0
390.0
390.0
390.0
390.0
390.0
390.0
390.0
391.0
391.0
391.0
391.0
391.0
391.0
391.0
391.0
391.0
391.0
392.0
392.0
392.0
392.0
392.0
392.0
392.0
392.0
392.0
392.0
393.0
393.0
393.

601.0
601.0
601.0
602.0
602.0
602.0
602.0
602.0
602.0
602.0
602.0
602.0
602.0
603.0
603.0
603.0
603.0
603.0
603.0
603.0
603.0
603.0
603.0
604.0
604.0
604.0
604.0
604.0
604.0
604.0
604.0
604.0
604.0
605.0
605.0
605.0
605.0
605.0
605.0
605.0
605.0
605.0
605.0
606.0
606.0
606.0
606.0
606.0
606.0
606.0
606.0
606.0
606.0
607.0
607.0
607.0
607.0
607.0
607.0
607.0
607.0
607.0
607.0
608.0
608.0
608.0
608.0
608.0
608.0
608.0
608.0
608.0
608.0
609.0
609.0
609.0
609.0
609.0
609.0
609.0
609.0
609.0
609.0
610.0
610.0
610.0
610.0
610.0
610.0
610.0
610.0
610.0
610.0
611.0
611.0
611.0
611.0
611.0
611.0
611.0
611.0
611.0
611.0
612.0
612.0
612.0
612.0
612.0
612.0
612.0
612.0
612.0
612.0
613.0
613.0
613.0
613.0
613.0
613.0
613.0
613.0
613.0
613.0
614.0
614.0
614.0
614.0
614.0
614.0
614.0
614.0
614.0
614.0
615.0
615.0
615.0
615.0
615.0
615.0
615.0
615.0
615.0
615.0
616.0
616.0
616.0
616.0
616.0
616.0
616.0
616.0
616.0
616.0
617.0
617.0
617.0
617.0
617.0
617.0
617.0
617.0
617.0
617.0
618.0
618.0
618.0
618.

837.0
837.0
837.0
837.0
837.0
837.0
837.0
837.0
837.0
837.0
838.0
838.0
838.0
838.0
838.0
838.0
838.0
838.0
838.0
838.0
839.0
839.0
839.0
839.0
839.0
839.0
839.0
839.0
839.0
839.0
840.0
840.0
840.0
840.0
840.0
840.0
840.0
840.0
840.0
840.0
841.0
841.0
841.0
841.0
841.0
841.0
841.0
841.0
841.0
841.0
842.0
842.0
842.0
842.0
842.0
842.0
842.0
842.0
842.0
842.0
843.0
843.0
843.0
843.0
843.0
843.0
843.0
843.0
843.0
843.0
844.0
844.0
844.0
844.0
844.0
844.0
844.0
844.0
844.0
844.0
845.0
845.0
845.0
845.0
845.0
845.0
845.0
845.0
845.0
845.0
846.0
846.0
846.0
846.0
846.0
846.0
846.0
846.0
846.0
846.0
847.0
847.0
847.0
847.0
847.0
847.0
847.0
847.0
847.0
847.0
848.0
848.0
848.0
848.0
848.0
848.0
848.0
848.0
848.0
848.0
849.0
849.0
849.0
849.0
849.0
849.0
849.0
849.0
849.0
849.0
850.0
850.0
850.0
850.0
850.0
850.0
850.0
850.0
850.0
850.0
851.0
851.0
851.0
851.0
851.0
851.0
851.0
851.0
851.0
851.0
852.0
852.0
852.0
852.0
852.0
852.0
852.0
852.0
852.0
852.0
853.0
853.0
853.0
853.0
853.0
853.0
853.

1050.0
1050.0
1050.0
1050.0
1050.0
1051.0
1051.0
1051.0
1051.0
1051.0
1051.0
1051.0
1051.0
1051.0
1051.0
1052.0
1052.0
1052.0
1052.0
1052.0
1052.0
1052.0
1052.0
1052.0
1052.0
1053.0
1053.0
1053.0
1053.0
1053.0
1053.0
1053.0
1053.0
1053.0
1053.0
1054.0
1054.0
1054.0
1054.0
1054.0
1054.0
1054.0
1054.0
1054.0
1054.0
1055.0
1055.0
1055.0
1055.0
1055.0
1055.0
1055.0
1055.0
1055.0
1055.0
1056.0
1056.0
1056.0
1056.0
1056.0
1056.0
1056.0
1056.0
1056.0
1056.0
1057.0
1057.0
1057.0
1057.0
1057.0
1057.0
1057.0
1057.0
1057.0
1057.0
1058.0
1058.0
1058.0
1058.0
1058.0
1058.0
1058.0
1058.0
1058.0
1058.0
1059.0
1059.0
1059.0
1059.0
1059.0
1059.0
1059.0
1059.0
1059.0
1059.0
1060.0
1060.0
1060.0
1060.0
1060.0
1060.0
1060.0
1060.0
1060.0
1060.0
1061.0
1061.0
1061.0
1061.0
1061.0
1061.0
1061.0
1061.0
1061.0
1061.0
1062.0
1062.0
1062.0
1062.0
1062.0
1062.0
1062.0
1062.0
1062.0
1062.0
1063.0
1063.0
1063.0
1063.0
1063.0
1063.0
1063.0
1063.0
1063.0
1063.0
1064.0
1064.0
1064.0
1064.0
1064.0
1064.0
1064.0
1064.0

...

1936.0
1936.0
1936.0
1936.0
1936.0
1936.0
1936.0
1936.0
1936.0
1937.0
1937.0
1937.0
1937.0
1937.0
1937.0
1937.0
1937.0
1937.0
1937.0
1938.0
1938.0
1938.0
1938.0
1938.0
1938.0
1938.0
1938.0
1938.0
1938.0
1939.0
1939.0
1939.0
1939.0
1939.0
1939.0
1939.0
1939.0
1939.0
1939.0
1940.0
1940.0
1940.0
1940.0
1940.0
1940.0
1940.0
1940.0
1940.0
1940.0
1941.0
1941.0
1941.0
1941.0
1941.0
1941.0
1941.0
1941.0
1941.0
1941.0
1942.0
1942.0
1942.0
1942.0
1942.0
1942.0
1942.0
1942.0
1942.0
1942.0
1943.0
1943.0
1943.0
1943.0
1943.0
1943.0
1943.0
1943.0
1943.0
1943.0
1944.0
1944.0
1944.0
1944.0
1944.0
1944.0
1944.0
1944.0
1944.0
1944.0
1945.0
1945.0
1945.0
1945.0
1945.0
1945.0
1945.0
1945.0
1945.0
1945.0
1946.0
1946.0
1946.0
1946.0
1946.0
1946.0
1946.0
1946.0
1946.0
1946.0
1947.0
1947.0
1947.0
1947.0
1947.0
1947.0
1947.0
1947.0
1947.0
1947.0
1948.0
1948.0
1948.0
1948.0
1948.0
1948.0
1948.0
1948.0
1948.0
1948.0
1949.0
1949.0
1949.0
1949.0
1949.0
1949.0
1949.0
1949.0
1949.0
1949.0
1950.0
1950.0
1950.0
1950.0

2148.0
2148.0
2148.0
2149.0
2149.0
2149.0
2149.0
2149.0
2149.0
2149.0
2149.0
2149.0
2149.0
2150.0
2150.0
2150.0
2150.0
2150.0
2150.0
2150.0
2150.0
2150.0
2150.0
2151.0
2151.0
2151.0
2151.0
2151.0
2151.0
2151.0
2151.0
2151.0
2151.0
2152.0
2152.0
2152.0
2152.0
2152.0
2152.0
2152.0
2152.0
2152.0
2152.0
2153.0
2153.0
2153.0
2153.0
2153.0
2153.0
2153.0
2153.0
2153.0
2153.0
2154.0
2154.0
2154.0
2154.0
2154.0
2154.0
2154.0
2154.0
2154.0
2154.0
2155.0
2155.0
2155.0
2155.0
2155.0
2155.0
2155.0
2155.0
2155.0
2155.0
2156.0
2156.0
2156.0
2156.0
2156.0
2156.0
2156.0
2156.0
2156.0
2156.0
2157.0
2157.0
2157.0
2157.0
2157.0
2157.0
2157.0
2157.0
2157.0
2157.0
2158.0
2158.0
2158.0
2158.0
2158.0
2158.0
2158.0
2158.0
2158.0
2158.0
2159.0
2159.0
2159.0
2159.0
2159.0
2159.0
2159.0
2159.0
2159.0
2159.0
2160.0
2160.0
2160.0
2160.0
2160.0
2160.0
2160.0
2160.0
2160.0
2160.0
2161.0
2161.0
2161.0
2161.0
2161.0
2161.0
2161.0
2161.0
2161.0
2161.0
2162.0
2162.0
2162.0
2162.0
2162.0
2162.0
2162.0
2162.0
2162.0
2162.0

2378.0
2378.0
2378.0
2378.0
2379.0
2379.0
2379.0
2379.0
2379.0
2379.0
2379.0
2379.0
2379.0
2379.0
2380.0
2380.0
2380.0
2380.0
2380.0
2380.0
2380.0
2380.0
2380.0
2380.0
2381.0
2381.0
2381.0
2381.0
2381.0
2381.0
2381.0
2381.0
2381.0
2381.0
2382.0
2382.0
2382.0
2382.0
2382.0
2382.0
2382.0
2382.0
2382.0
2382.0
2383.0
2383.0
2383.0
2383.0
2383.0
2383.0
2383.0
2383.0
2383.0
2383.0
2384.0
2384.0
2384.0
2384.0
2384.0
2384.0
2384.0
2384.0
2384.0
2384.0
2385.0
2385.0
2385.0
2385.0
2385.0
2385.0
2385.0
2385.0
2385.0
2385.0
2386.0
2386.0
2386.0
2386.0
2386.0
2386.0
2386.0
2386.0
2386.0
2386.0
2387.0
2387.0
2387.0
2387.0
2387.0
2387.0
2387.0
2387.0
2387.0
2387.0
2388.0
2388.0
2388.0
2388.0
2388.0
2388.0
2388.0
2388.0
2388.0
2388.0
2389.0
2389.0
2389.0
2389.0
2389.0
2389.0
2389.0
2389.0
2389.0
2389.0
2390.0
2390.0
2390.0
2390.0
2390.0
2390.0
2390.0
2390.0
2390.0
2390.0
2391.0
2391.0
2391.0
2391.0
2391.0
2391.0
2391.0
2391.0
2391.0
2391.0
2392.0
2392.0
2392.0
2392.0
2392.0
2392.0
2392.0
2392.0
2392.0

2601.0
2601.0
2601.0
2601.0
2601.0
2601.0
2601.0
2601.0
2601.0
2602.0
2602.0
2602.0
2602.0
2602.0
2602.0
2602.0
2602.0
2602.0
2602.0
2603.0
2603.0
2603.0
2603.0
2603.0
2603.0
2603.0
2603.0
2603.0
2603.0
2604.0
2604.0
2604.0
2604.0
2604.0
2604.0
2604.0
2604.0
2604.0
2604.0
2605.0
2605.0
2605.0
2605.0
2605.0
2605.0
2605.0
2605.0
2605.0
2605.0
2606.0
2606.0
2606.0
2606.0
2606.0
2606.0
2606.0
2606.0
2606.0
2606.0
2607.0
2607.0
2607.0
2607.0
2607.0
2607.0
2607.0
2607.0
2607.0
2607.0
2608.0
2608.0
2608.0
2608.0
2608.0
2608.0
2608.0
2608.0
2608.0
2608.0
2609.0
2609.0
2609.0
2609.0
2609.0
2609.0
2609.0
2609.0
2609.0
2609.0
2610.0
2610.0
2610.0
2610.0
2610.0
2610.0
2610.0
2610.0
2610.0
2610.0
2611.0
2611.0
2611.0
2611.0
2611.0
2611.0
2611.0
2611.0
2611.0
2611.0
2612.0
2612.0
2612.0
2612.0
2612.0
2612.0
2612.0
2612.0
2612.0
2612.0
2613.0
2613.0
2613.0
2613.0
2613.0
2613.0
2613.0
2613.0
2613.0
2613.0
2614.0
2614.0
2614.0
2614.0
2614.0
2614.0
2614.0
2614.0
2614.0
2614.0
2615.0
2615.0
2615.0
2615.0

2843.0
2844.0
2844.0
2844.0
2844.0
2844.0
2844.0
2844.0
2844.0
2844.0
2844.0
2845.0
2845.0
2845.0
2845.0
2845.0
2845.0
2845.0
2845.0
2845.0
2845.0
2846.0
2846.0
2846.0
2846.0
2846.0
2846.0
2846.0
2846.0
2846.0
2846.0
2847.0
2847.0
2847.0
2847.0
2847.0
2847.0
2847.0
2847.0
2847.0
2847.0
2848.0
2848.0
2848.0
2848.0
2848.0
2848.0
2848.0
2848.0
2848.0
2848.0
2849.0
2849.0
2849.0
2849.0
2849.0
2849.0
2849.0
2849.0
2849.0
2849.0
2850.0
2850.0
2850.0
2850.0
2850.0
2850.0
2850.0
2850.0
2850.0
2850.0
2851.0
2851.0
2851.0
2851.0
2851.0
2851.0
2851.0
2851.0
2851.0
2851.0
2852.0
2852.0
2852.0
2852.0
2852.0
2852.0
2852.0
2852.0
2852.0
2852.0
2853.0
2853.0
2853.0
2853.0
2853.0
2853.0
2853.0
2853.0
2853.0
2853.0
2854.0
2854.0
2854.0
2854.0
2854.0
2854.0
2854.0
2854.0
2854.0
2854.0
2855.0
2855.0
2855.0
2855.0
2855.0
2855.0
2855.0
2855.0
2855.0
2855.0
2856.0
2856.0
2856.0
2856.0
2856.0
2856.0
2856.0
2856.0
2856.0
2856.0
2857.0
2857.0
2857.0
2857.0
2857.0
2857.0
2857.0
2857.0
2857.0
2857.0
2858.0
2858.0